<a href="https://colab.research.google.com/github/PerlaSaiVarshithReddy/MyPortfolioCodewith/blob/main/Task_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

# Load Titanic dataset
df = pd.read_csv("/content/sample_data/netflix_titles.csv")

# Preview
df.head()


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [7]:
df.isnull().sum()


,0
show_id,0
type,0
title,0
director,2634
cast,825
country,831
date_added,10
release_year,0
rating,4
duration,3


In [8]:
# Fill missing 'director', 'cast', 'country' with 'Unknown'
df['director'].fillna('Unknown', inplace=True)
df['cast'].fillna('Unknown', inplace=True)
df['country'].fillna('Unknown', inplace=True)


/tmp/ipython-input-2247461696.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['director'].fillna('Unknown', inplace=True)
/tmp/ipython-input-2247461696.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usin

In [9]:
from sklearn.preprocessing import LabelEncoder

# Encode 'type' (TV Show = 0, Movie = 1)
df['type'] = df['type'].map({'TV Show': 0, 'Movie': 1})

# Encode 'rating' with LabelEncoder
le = LabelEncoder()
df['rating'] = le.fit_transform(df['rating'].astype(str))

# Simplify 'country' to top 5 countries, rest as 'Other'
top_countries = df['country'].value_counts().nlargest(5).index
df['country'] = df['country'].apply(lambda x: x if x in top_countries else 'Other')

# One-hot encode 'country'
df = pd.get_dummies(df, columns=['country'], drop_first=True)


In [10]:
from sklearn.preprocessing import StandardScaler
import numpy as np

# Separate numeric part of 'duration'
def convert_duration(val):
    if 'min' in str(val):
        return int(val.split()[0])
    else:
        return np.nan  # For TV Shows ("1 Season", etc.)

df['duration_minutes'] = df['duration'].apply(convert_duration)

# Fill missing durations with median (TV Shows will be NaN)
median_duration = df['duration_minutes'].median()
df['duration_minutes'].fillna(median_duration, inplace=True)

# Scale duration
scaler = StandardScaler()
df['duration_scaled'] = scaler.fit_transform(df[['duration_minutes']])


/tmp/ipython-input-45983288.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['duration_minutes'].fillna(median_duration, inplace=True)


In [11]:
df['release_decade'] = (df['release_year'] // 10) * 10


In [12]:
df['num_cast_members'] = df['cast'].apply(lambda x: len(str(x).split(',')) if x != 'Unknown' else 0)


In [13]:
df['has_multiple_directors'] = df['director'].apply(lambda x: 1 if ',' in x else 0)


In [14]:
selected_features = [
    'type',
    'rating',
    'duration_scaled',
    'release_year',
    'release_decade',
    'num_cast_members',
    'has_multiple_directors',
] + [col for col in df.columns if col.startswith("country_")]

df_model = df[selected_features]
df_model.head()


,type,rating,duration_scaled,release_year,release_decade,num_cast_members,has_multiple_directors,country_Japan,country_Other,country_United Kingdom,country_United States,country_Unknown
0,1,7,-0.385355,2020,2020,0,0,False,False,False,True,False
1,0,11,-0.046485,2021,2020,19,0,False,True,False,False,False
2,0,11,-0.046485,2021,2020,9,0,False,False,False,False,True
3,0,11,-0.046485,2021,2020,0,0,False,False,False,False,True
4,0,11,-0.046485,2021,2020,8,0,False,False,False,False,False
